#  Run Pix2Pix
This notebook is optimized for using pytorch (look at the environment on the top right). <br>
This is the main folder path: [~/ml/](http://localhost:8888/tree/ml)<br>
Image dataset is located here: [~/ml/dataset_oct_histology/](http://localhost:8888/tree/ml/dataset_oct_histology)<br>
[Github Link](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix)<br>
<br>
## Install

In [9]:
# Set up general varibles
root_path = '~/ml/'
dataset_path = root_path + 'dataset_oct_histology/'
code_main_folder = root_path + 'pix2pix_and_CycleGAN/'

# Install environment dependencies
!pip install --upgrade pip
!pip install opencv-python

# Clear code, start a fresh copy
import shutil
import os
if os.path.isdir(os.path.expanduser(code_main_folder)):
    shutil.rmtree(os.path.expanduser(code_main_folder))
    
# Get main model
!git clone --single-branch https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix {code_main_folder}
!pip install -r {code_main_folder}requirements.txt

Requirement already up-to-date: pip in /home/ubuntu/anaconda3/envs/aws_neuron_pytorch_p36/lib/python3.6/site-packages (20.1.1)
Cloning into '/home/ubuntu/ml/pix2pix_and_CycleGAN'...
remote: Enumerating objects: 2262, done.
remote: Total 2262 (delta 0), reused 0 (delta 0), pack-reused 2262
Receiving objects: 100% (2262/2262), 8.06 MiB | 26.88 MiB/s, done.
Resolving deltas: 100% (1456/1456), done.


## Build Dataset
This library requires OCT and histology images to be paired together.<br>
Code below merges images from  [patches_256px_256px](http://localhost:8888/tree/ml/dataset_oct_histology/patches_256px_256px) to [patches_256px_256px_combined](http://localhost:8888/tree/ml/dataset_oct_histology/patches_256px_256px_combined)

In [2]:
import image_combine_aux_functions

# Define train & test patches folders
train_patches_folder = dataset_path + 'patches_256px_256px_aspect_ratio_2_1/'
train_patches_folder_combined = train_patches_folder[:-1] + '_combined/'
test_patches_folder = dataset_path + 'patches_256px_256px_aspect_ratio_2_1/'
test_patches_folder_combined = test_patches_folder[:-1] + '_combined/'

# Generate combine train dataset of A and B
image_combine_aux_functions.combine_images(
    img_fold_A = (train_patches_folder + 'train_A/'),
    img_fold_B = (train_patches_folder + 'train_B/'),
    img_fold_AB = (train_patches_folder_combined + 'train/'))

# Make sure that in train_patches_folder_combined there is no test folder.
# This prevents mistake leakage of data.
image_combine_aux_functions.clear_dir(train_patches_folder_combined + 'test/')

## Train
Run code below to train model.<br>
Results can be viewed here: [~/ml/checkpoints/pix2pix_2to1ratio/web/index.html](http://localhost:8888/view/ml/checkpoints/pix2pix_2to1ratio/web/index.html) as the model trains.<br>

In [3]:
# Default setting includes flip which trains on left-right flips as well.
#'--preprocess crop' allows user to load larger than 256X256 images and just randomly crop the right size for training.
#  this is not as recommended because most of the crops will be just black with no information
# If model is stuck, restart using --continue_train --epoch_count <number> to get numbering right.
!python {code_main_folder}train.py --name pix2pix_2to1ratio --dataroot {train_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /home/ubuntu/ml/checkpoints   	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /home/ubuntu/ml/dataset_oct_histology/patches_256px_256px_aspect_ratio_2_1_combined/	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                     



Could not connect to Visdom server. 
 Trying to start a server....
Command: /home/ubuntu/anaconda3/envs/aws_neuron_pytorch_p36/bin/python -m visdom.server -p 8097 &>/dev/null &
create web directory /home/ubuntu/ml/checkpoints/pix2pix_2to1ratio/web...
(epoch: 1, iters: 100, time: 0.100, data: 0.639) G_GAN: 2.457 G_L1: 24.144 D_real: 0.048 D_fake: 0.522 
(epoch: 1, iters: 200, time: 0.099, data: 0.001) G_GAN: 3.370 G_L1: 21.292 D_real: 0.078 D_fake: 0.195 
End of epoch 1 / 200 	 Time Taken: 14 sec
learning rate = 0.0002000
(epoch: 2, iters: 89, time: 0.101, data: 0.001) G_GAN: 3.019 G_L1: 23.632 D_real: 0.030 D_fake: 0.195 
(epoch: 2, iters: 189, time: 0.299, data: 0.001) G_GAN: 2.623 G_L1: 22.070 D_real: 0.157 D_fake: 0.156 
End of epoch 2 / 200 	 Time Taken: 12 sec
learning rate = 0.0002000
(epoch: 3, iters: 78, time: 0.101, data: 0.001) G_GAN: 3.097 G_L1: 17.196 D_real: 0.056 D_fake: 0.399 
(epoch: 3, iters: 178, time: 0.101, data: 0.001) G_GAN: 2.881 G_L1: 32.487 D_real: 0.051 D_fa

## Test

Main test results can be viewed here: [~/ml/results/pix2pix_2to1ratio/test_latest/index.html](http://localhost:8888/view/ml/results/pix2pix_2to1ratio/test_latest/index.html) after test command

Latent space values are visible here: [~/ml/results/pix2pix_2to1ratio/feats/center/](http://localhost:8888/tree/ml/results/pix2pix_2to1ratio/feats/center/)


In [ ]:
# Generate combine train dataset of A and B
image_combine_aux_functions.combine_images(
    img_fold_A = (test_patches_folder + 'test_A/'),
    img_fold_B = (test_patches_folder + 'test_B/'),
    img_fold_AB = (test_patches_folder_combined + 'test/'))

# Copy code patch to generate latent space
shutil.copy('networks.py',os.path.expanduser(code_main_folder + "models/"))
shutil.copy('pix2pix_model.py',os.path.expanduser(code_main_folder + "models/"))

# Main test results + weights at the center
#  --preprocess none allows to directly process a non 256x256 images using the convolutional proprety of the network
!python {code_main_folder}test.py --name pix2pix_2to1ratio --dataroot {test_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints --results_dir {root_path}results

# Generate images and features on the train dataset as well
!python {code_main_folder}test.py --name pix2pix_2to1ratio --dataroot {test_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints --results_dir {root_path}results --phase train